In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/uncertainty-aware-llms')

In [4]:
!pip install -q accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 25.5 MB/s eta 0:00:00


## Import packages for finetuning


In [6]:
import torch
from datasets import load_dataset
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig,PeftModel
from trl import SFTTrainer

## Hugging Face Login

In [7]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `Uncertainity` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-

## Loading the base model and dataset

In [8]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
datset_name = "suyog-ghimire/UncertaintyQA"
new_model = "llama-3.2-1B-Uncertainity"

## QLoRA parameters

In [9]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

## Bits and Bytes parameters

In [11]:
use_4bit = True # Activate 4-bit precision base model loading
bnb_4bit_compute_dtype = "float16"  # computations are done in F16
bnb_4bit_quant_type = "nf4" # Choosing Quantization type (fp4 or nf4)
use_nested_quant = False